In [ ]:
import t3f
import tensorflow as tf
import numpy as np
from environments import Snake
from methods import ReplayMemory
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def get_actions(s, qtt, sess):
    feed_dict = {input_states_:s}
    return sess.run(q_select, feed_dict=feed_dict)

def tf_repeat(x, num):
    u = tf.reshape(x, (-1, 1))
    u = tf.tile(u, tf.TensorShape([tf.Dimension(1), num]))
    u = tf.reshape(u, (-1, ))
    return u

def get_q_values(qtt, s):
    # s [None x w x h]
    
    s_shape = s.shape[1:].as_list()
    
    b = s.shape[0]
    w = s_shape[0]
    h = s_shape[1]
    s_idx = tf.reshape(s, (-1, h * w))    
    s_a_idx = tf.concat(4 * [s_idx], axis=0) 
    a_idx = tf.reshape(tf_repeat(tf.range(start=0, limit=4), b), (-1, 1)) 
    s_a_idx = tf.concat([s_a_idx, a_idx], axis=1)
    vals = t3f.gather_nd(qtt, s_a_idx, dtype=tf.float32)
    return tf.transpose(tf.reshape(vals, shape=(4, b)))


def get_q_action_values(qtt, sa):
    return t3f.gather_nd(qtt, sa, dtype=tf.float32)

In [ ]:
w = 4
h = 4

batch_size = 512

replay_memory_size=50000
replay_start_size=10000
init_eps=1
final_eps=0.1
annealing_steps=100000
gamma=0.99
max_episode_length=400
num_episodes = 1000000

In [ ]:
train_env = Snake(grid_size=(w, h))

In [ ]:
state_shape = (w * h) * [4, ]

In [6]:
shape = (w * h) * [4,] + [4, ]

In [7]:
q0init = t3f.cast(t3f.random_tensor(shape=shape, stddev=0.001, tt_rank=8), tf.float32)

In [8]:
q0 = t3f.get_variable('Q', initializer=q0init)

In [9]:
input_states = tf.placeholder(tf.int32, [batch_size, w, h, 1])
input_actions = tf.placeholder(tf.int32, [batch_size, 1])
input_rewards = tf.placeholder(tf.float32, [batch_size, ])

input_states_ = tf.placeholder(tf.int32, [batch_size, w, h, 1])


In [10]:
input_s_and_a = tf.concat([tf.reshape(input_states, (-1, w * h)), input_actions], axis=1) 

In [11]:
q = get_q_action_values(q0, input_s_and_a)

In [12]:
input_s_ = tf.reshape(input_states_, (-1, w, h))

q_ = tf.reduce_max(get_q_values(q0, input_s_), axis=1)

q_select = tf.argmax(get_q_values(q0, input_s_), axis=1)

In [13]:
targets = gamma * q_ + input_rewards


In [14]:
loss = tf.losses.huber_loss(q, targets)

In [15]:
optimizer = tf.train.AdamOptimizer(2.5e-4)

step = optimizer.minimize(loss)

In [16]:
rep_buffer = ReplayMemory(replay_memory_size)
frame_count = 0
while (frame_count < replay_start_size):
    s = train_env.reset()
    for time_step in range(max_episode_length):
        a = np.random.randint(4)
        s_, r, end = train_env.step(a)
        rep_buffer.push(s, a, np.sign(r), s_, end)
        s = s_
        frame_count += 1
        if end:
            break

In [22]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"] = str(2)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [23]:
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

In [24]:
s = train_env.reset()

In [ ]:
train_rewards = []
for i in range(num_episodes):
    s = train_env.reset()
    R = 0
    for time_step in range(max_episode_length):
        a = get_actions([s] * batch_size, q0, sess)[0]
        s_, r, done = train_env.step(a)
        R += r
        if r == 0: r = -0.01
        rep_buffer.push(s, a, r, s_, end)
        s = s_

        if time_step % 4 == 0:
            batch = rep_buffer.get_batch(batch_size)
            sess.run(step, feed_dict={input_states:batch.s,
                                      input_actions:batch.a.reshape((-1, 1)),
                                      input_rewards:batch.r,
                                      input_states_:batch.s_})
        if done: break
    train_rewards.append(R) 
    if i % 1000 == 0:
        avg_reward = np.mean(train_rewards[-1000:])
        max_reward = np.max(train_rewards[-1000:])
        print("Train info:", avg_reward, max_reward) 

Train info: -1.0 -1
Train info: -0.77 2
Train info: -0.563 5
Train info: 0.495 5


In [ ]:
rep_buffer.get_batch(32).s_.shape